# EDA Basic Lesson Plan

Welcome to the Exploratory Data Analysis (EDA) lesson! This notebook is designed to guide you through the essential steps of taking a raw dataset, understanding its structure, cleaning it, and transforming it for analysis.

**Structure:**
* **Section 1: The First Look:** How to read files and get a quick statistical summary.
* **Section 2: The Cleanup:** Handling the messy reality of data (missing values, duplicates, outliers).
* **Section 3: Refinement & Transformation:** preparing data for machine learning or deep analysis (mapping, string manipulation).

**For Instructors:** Use the narrative to introduce concepts before running code cells.
**For Learners:** Read the comments in the code to understand exactly what each parameter does.

### Setup
As always, we start by importing the necessary libraries. `pandas` is our primary tool for data manipulation, and `numpy` helps with numerical operations.

In [42]:
import pandas as pd
import numpy as np

---

# Section 1: The First Look: Inspection & Summary

**Goal:** Load data from different file formats and get a "health check" of the data using descriptive statistics.

We will cover:
1. Reading CSV and Text files (handling headers, delimiters, and special characters).
2. Summarizing data using `.describe()`, `.sum()`, and `.value_counts()`.

## Part 1: Reading Data

Pandas is incredibly flexible with input formats. Let's start by reading a standard Comma-Separated Values (CSV) file.

In [1]:
# Let's inspect the raw file content first using a shell command
!cat ../data/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [4]:
# Use read_csv to load the data into a DataFrame
# By default, it assumes the first row is the header
df = pd.read_csv("../data/ex1.csv")

df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**Scenario:** What if the file doesn't have a header row? If we don't specify this, pandas will mistakenly use the first row of data as column names.

In [5]:
!cat ../data/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [6]:
# Option 1: Tell pandas there is no header. It will assign integers (0, 1, 2...) as column names.
pd.read_csv("../data/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
# Option 2: Provide your own column names using the 'names' parameter
pd.read_csv("../data/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**Indexing:** You can also designate a specific column to be the index (row labels) of the DataFrame, rather than the default 0, 1, 2... index.

In [8]:
names = ["a", "b", "c", "d", "message"]

# Use 'index_col' to set the 'message' column as the index
pd.read_csv("../data/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


**Irregular Separators:** Sometimes data isn't separated by commas. It might be tabs, spaces, or a variable amount of whitespace. We can use Regular Expressions (Regex) to handle this.

In [12]:
# Inspect a file with messy whitespace
!cat ../data/ex3.txt

A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491

In [48]:
# sep="\s+" is a regex that means "one or more whitespace characters"
result = pd.read_csv("../data/ex3.txt", sep="\s+")

result

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_1876/3874157656.py:2: SyntaxWarning: invalid escape sequence '\s'
  result = pd.read_csv("../data/ex3.txt", sep="\s+")


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


**Skipping Rows:** Sometimes files contain comments or metadata at the top that we want to ignore.

In [15]:
!cat ../data/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [17]:
# Skip specific rows by index (0, 2, and 3 here) to get to the clean data
pd.read_csv("../data/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**Handling Missing Values (at load time):** Pandas is smart about identifying missing data (empty strings, 'NA', 'NULL'), but we can also define our own "sentinels" for missing values.

In [18]:
!cat ../data/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,NULL

In [19]:
# Default behavior: reads 'NA' and 'NULL' as NaN (Not a Number/Missing)
result = pd.read_csv("../data/ex5.csv")

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [20]:
# Verify where the missing values are detected
result.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,True


You can customize what is considered "Missing" for each column specifically.

In [21]:
# Define a dictionary of sentinels
# For column 'message', treat 'foo' and 'NA' as null
# For column 'something', treat 'two' as null
sentinels = {"message": ["NULL", "NA"], "something": ["two"]}

pd.read_csv("../data/ex5.csv", na_values=sentinels, keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


**Reading Excel:** Reading Excel files works similarly, but we can specify sheet names.

In [45]:
# Load the Excel file object
xlsx = pd.ExcelFile("../data/Resaleflatpricesbasedonregistrationdate.xlsx")

# List available sheets
xlsx.sheet_names

['2017']

In [46]:
# Parse a specific sheet into a DataFrame
xlsx.parse(sheet_name="2017")

,month,town,flat_type,block,street_name,...,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,...,44,Improved,1979,61 years 04 months,232000
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,...,67,New Generation,1978,60 years 07 months,250000
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,...,67,New Generation,1980,62 years 05 months,262000
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,...,68,New Generation,1980,62 years 01 month,265000
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,...,67,New Generation,1980,62 years 05 months,265000
...,...,...,...,...,...,...,...,...,...,...,...
1180,2017-01,YISHUN,EXECUTIVE,387,YISHUN RING RD,...,142,Apartment,1988,70 years 05 months,587000
1181,2017-01,YISHUN,EXECUTIVE,877,YISHUN ST 81,...,142,Apartment,1987,69 years 11 months,590000
1182,2017-01,YISHUN,EXECUTIVE,836,YISHUN ST 81,...,146,Maisonette,1988,70 years 01 month,673000
1183,2017-01,YISHUN,EXECUTIVE,633A,YISHUN ST 61,...,164,Apartment,1992,74 years 05 months,678000


In [47]:
# Shortcut: Read directly without creating an ExcelFile object first
frame = pd.read_excel("../data/Resaleflatpricesbasedonregistrationdate.xlsx", sheet_name="2017")

frame

,month,town,flat_type,block,street_name,...,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,...,44,Improved,1979,61 years 04 months,232000
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,...,67,New Generation,1978,60 years 07 months,250000
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,...,67,New Generation,1980,62 years 05 months,262000
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,...,68,New Generation,1980,62 years 01 month,265000
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,...,67,New Generation,1980,62 years 05 months,265000
...,...,...,...,...,...,...,...,...,...,...,...
1180,2017-01,YISHUN,EXECUTIVE,387,YISHUN RING RD,...,142,Apartment,1988,70 years 05 months,587000
1181,2017-01,YISHUN,EXECUTIVE,877,YISHUN ST 81,...,142,Apartment,1987,69 years 11 months,590000
1182,2017-01,YISHUN,EXECUTIVE,836,YISHUN ST 81,...,146,Maisonette,1988,70 years 01 month,673000
1183,2017-01,YISHUN,EXECUTIVE,633A,YISHUN ST 61,...,164,Apartment,1992,74 years 05 months,678000


## Part 2: Summarizing and Computing Descriptive Statistics

Once data is loaded, we need to understand its "shape". We use summary statistics to get a birds-eye view.

> **📊 Visual Illustration Available**: See ![descriptive_statistics](../visualisations/01_descriptive_statistics.png) for visual representations of descriptive statistics including histograms, box plots, and how statistics like mean, median, and quartiles relate to data distributions.


Let's create a sample DataFrame with some missing values to test our summary methods.

In [32]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5], [np.nan, np.nan], [0.75, -1.3]],
                  index=["a", "b", "c", "d"], columns=["one", "two"])

df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


**Reductions:** Methods like `.sum()` or `.mean()` reduce a Series of numbers to a single value. By default, they operate down the rows (`axis=0`).

In [33]:
# Sums down the rows (returns sum for each column)
df.sum()

one    9.25
two   -5.80
dtype: float64

In [34]:
# Sums across the columns (returns sum for each row)
# axis=1 is synonymous with axis='columns'
df.sum(axis="columns")

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

**Handling NAs in Calculations:** By default, pandas ignores NaNs (treats them as zero for sums). We can change this with `skipna`.

In [35]:
# If any value is NaN, the result is NaN
df.sum(skipna=False)

one   NaN
two   NaN
dtype: float64

In [36]:
df.sum(axis=1, skipna=False)

a     NaN
b    2.60
c     NaN
d   -0.55
dtype: float64

**Indirect Statistics:** Finding *where* the max or min value is located (the index label).

In [37]:
# Returns the index of the maximum value
df.idxmax()

one    b
two    d
dtype: object

In [38]:
# Returns the index of the minimum value
df.idxmin()

one    d
two    b
dtype: object

**Accumulations:** Computing cumulative sums.

In [39]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


**The `.describe()` method:** This is your best friend for a quick snapshot of numeric data.

In [40]:
# Provides count, mean, std, min, quartiles, and max
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


**Categorical/Non-Numeric Data:** `.describe()` behaves differently for string data, showing counts and uniqueness.

In [ ]:
obj = pd.Series(["c", "a", "d", "a", "b", "b", "c", "c"])

# For object data, we get count, unique, top (most frequent), and freq
obj.describe()

In [ ]:
# Get unique values
obj.unique()

In [ ]:
# Get frequency counts of each unique value
obj.value_counts()

**Exercise:** Sort value counts in ascending order.

In [ ]:
obj.value_counts().sort_index()

---

# Section 2: The Cleanup: Missing Data, Duplicates & Outliers

**Goal:** Real-world data is dirty. In this section, we will learn techniques to identify and fix common quality issues.

We will cover:
1. **Missing Data:** Detecting nulls and deciding whether to drop them or fill them.
2. **Duplicates:** Finding and removing repeated records.
3. **Outliers:** identifying extreme values using boolean indexing.

## Part 1: Handling Missing Data

Missing data is often represented as `NaN` (Not a Number) or `None`.

> **📊 Visual Illustration Available**: See ![missing_data](../visualisations/03_missing_data.png) for visualizations of missing data patterns, heatmaps, and comparison of different strategies for handling missing values.


In [ ]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])

float_data

In [ ]:
# .isna() returns a boolean mask (True if missing, False if present)
float_data.isna()

The built-in Python `None` value is also treated as NA in pandas object arrays.

In [ ]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

string_data

In [ ]:
string_data.isna()

### Strategy 1: Dropping Missing Data (`dropna`)
The simplest strategy is to just remove the rows or columns that contain missing values.

In [ ]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])

# Removes all NaN values from the Series
data.dropna()

In [ ]:
# This is equivalent to boolean filtering:
data[data.notna()]

With DataFrames, `dropna` by default drops **any row** containing **any missing value**.

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan], 
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])

data

In [ ]:
# Drops rows 1, 2, and 3 because they have at least one NaN
data.dropna()

We can control this behavior. `how='all'` only drops rows where **all** values are NaN.

In [ ]:
data.dropna(how="all")

To drop **columns** instead of rows, pass `axis=1`.

In [ ]:
# Let's add a column of all NaNs first
data[4] = np.nan

data

In [ ]:
# Drops the column '4' because it is all NaNs
data.dropna(axis="columns", how="all")

We can also set a **threshold**: keep only rows containing at least `n` valid observations.

In [ ]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
# Set some missing values
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan

df

In [ ]:
df.dropna()

In [ ]:
# Keep rows that have at least 2 non-NaN values
df.dropna(thresh=2)

### Strategy 2: Filling Missing Data (`fillna`)
Instead of losing data, we can fill the holes with a constant or a calculated value.

In [ ]:
# Replace all NaNs with 0
df.fillna(0)

You can specify different fill values for each column:

In [ ]:
# Fill column 1 with 0.5, and column 2 with 0
df.fillna({1: 0.5, 2: 0})

**Forward/Backward Fill:** Useful for time-series data, where you propagate the last valid observation forward or backward.

In [ ]:
# Propagate next valid value backward to fill gaps
df.fillna(method="bfill")

In [ ]:
# Same, but limit how many rows are filled consecutively
df.fillna(method="bfill", limit=2)

**Imputation:** Filling with the mean or median is a very common technique.

In [ ]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])

data

In [ ]:
# Fill with the mean of the available data
data.fillna(data.mean())

### Exercise: Handling Missing Data
Practice using these methods on the dataframe below.

In [ ]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

df.iloc[[2,4,5], 1] = np.nan
df.iloc[4:, 2] = np.nan
df.iloc[3:5, 0] = np.nan

df

> **Tasks:**
> 1. Remove rows with *any* missing values.
> 2. Remove rows with *all* missing values.
> 3. Fill missing values with forward fill.
> 4. Fill missing values with mean of the column.

## Part 2: Handling Duplicates

Duplicate rows can skew analysis and models. We typically identify them and remove them.

> **📊 Visual Illustration Available**: See ![duplicates](../visualisations/06_duplicates.png) for visualizations showing how duplicates affect datasets and the impact of different `keep` parameter options.


In [49]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


`duplicated()` returns a boolean Series indicating whether each row has been seen before.

In [50]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

`drop_duplicates()` creates a new DataFrame with the duplicates removed.

In [51]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


**Subset:** Sometimes we only care about duplicates in specific columns.

In [52]:
data["v1"] = range(7)

data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [53]:
# Drop duplicates considering only column 'k1'
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


**Keep:** By default, it keeps the first occurrence. We can keep the last one instead.

In [54]:
data.drop_duplicates(subset=["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Part 3: Handling Outliers

Outliers are extreme values that deviate significantly from the rest of the data. We can filter them using boolean indexing.

> **📊 Visual Illustration Available**: See ![outliers](../visualisations/04_outliers.png) for visualizations of outlier detection methods (IQR, Z-score) and comparison of different handling strategies.


In [55]:
# Create a dataset with normal distribution
data = pd.DataFrame(np.random.standard_normal((1000, 4)))

data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.016547,-0.010351,-0.034462,-0.019750
std,1.020401,0.985568,0.986518,0.973306
min,-3.532323,-3.284042,-3.340144,-2.915789
25%,-0.644493,-0.662546,-0.664107,-0.666466
50%,-0.014816,0.012678,-0.050222,-0.018153
75%,0.735979,0.647574,0.596490,0.640456
max,2.887252,3.151708,2.893492,2.935141


**Detection:** Let's find values exceeding 3 in absolute value (Standard Deviation > 3).

In [56]:
col = data[2]

# Boolean indexing to find rows where absolute value > 3
col[col.abs() > 3]

47    -3.209424
93    -3.059522
450   -3.141597
521   -3.340144
Name: 2, dtype: float64

To find **any row** that has an outlier in **any column**, we use `.any(axis=1)`.

In [57]:
# 1. data.abs() > 3 returns a boolean DataFrame
# 2. .any(axis="columns") checks if any value in the row is True
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
47,1.721930,-0.848886,-3.209424,-0.960137
93,-2.298101,-0.563325,-3.059522,-0.877793
320,0.395326,-3.066994,0.428549,-0.185271
450,0.582265,0.673486,-3.141597,0.194761
519,0.005244,-3.284042,1.776802,-1.658930
521,0.055849,0.170041,-3.340144,-0.327332
540,-3.532323,-0.927555,2.278419,-2.132859
968,-1.156873,3.151708,-0.312158,1.563472


**Capping:** Instead of removing outliers, we can cap them at a threshold.

In [61]:
# Set values > 3 to 3, and < -3 to -3, preserving the sign
data[data.abs() > 3] = np.sign(data) * 3

In [62]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.017080,-0.010152,-0.033712,-0.019750
std,1.018685,0.983982,0.984163,0.973306
min,-3.000000,-3.000000,-3.000000,-2.915789
25%,-0.644493,-0.662546,-0.664107,-0.666466
50%,-0.014816,0.012678,-0.050222,-0.018153
75%,0.735979,0.647574,0.596490,0.640456
max,2.887252,3.000000,2.893492,2.935141


**Removal:** Or we can just drop the rows with outliers.

In [60]:
# Keep rows where ALL columns are within the threshold ( < 3)
data[(data.abs() < 3).all(axis="columns")]

,0,1,2,3
0,-1.174174,0.351307,-0.528487,-0.691595
1,1.307583,0.188710,0.985721,-0.382568
2,-0.820290,-0.710395,-0.187377,0.671889
3,-1.393125,0.688506,-0.658621,-1.334760
4,0.238524,-0.688735,-1.192319,0.573608
...,...,...,...,...
995,0.793943,-0.698478,0.303620,-2.324317
996,1.257269,0.119404,-1.352108,0.525039
997,-0.444556,0.129735,0.717424,-0.484108
998,1.377021,-1.127268,1.169590,0.022725


---

# Section 3: Refinement & Transformation

**Goal:** Now that the data is clean, we need to transform it into the right format for analysis.

We will cover:
1.  **Mapping & Replacing:** Changing values based on a dictionary logic.
2.  **String Manipulation:** Cleaning text data using the `.str` accessor.
3.  **Exporting:** Saving your hard work to a file.

## Part 1: Transforming Data (Mapping)

Sometimes we need to add new columns based on existing ones. `map()` is perfect for this.

In [63]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon", "pastrami", 
                                "corned beef", "bacon", "pastrami", "honey ham", 
                                "nova lox"],
                         "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


**Scenario:** We want to add a column indicating the animal source of each food.

In [64]:
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salmon"
}

In [65]:
# .map() looks up the value in the 'food' column in our dictionary
data["animal"] = data["food"].map(meat_to_animal)

data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


You can also pass a function to `map()` for custom logic.

In [66]:
def get_animal(x):
    return meat_to_animal[x]

data["food"].map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

**Replacing Values:** `replace` is a specialized version of map, great for fixing sentinel values (like -999 for missing data).

In [67]:
data = pd.Series([1, -999, 2, -999, -1000, 3])

data

0       1
1    -999
2       2
3    -999
4   -1000
5       3
dtype: int64

In [68]:
# Replace -999 with NaN
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [69]:
# Replace multiple values at once
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [70]:
# Replace with different values (-999 -> NaN, -1000 -> 0)
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [71]:
# Using a dictionary for clarity
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Exercise: Transforming Data

Try replacing values in a DataFrame.

In [72]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

df.iloc[2:, 1] = -999
df.iloc[4:, 2] = 999

df

,0,1,2
0,0.286504,0.902576,0.545338
1,0.570005,-1.686979,-1.493998
2,0.293649,-999.000000,1.609886
3,1.510463,-999.000000,0.551710
4,1.093413,-999.000000,999.000000
5,0.286856,-999.000000,999.000000


> **Task:** Replace -999 with NaN and 999 with 0.

## Part 2: String Manipulation

Pandas has a special accessor `.str` that unlocks string methods for an entire Series at once. This handles missing values gracefully.

In [73]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}

data = pd.Series(data)

data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [74]:
# Check if 'gmail' exists in each string
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Note on Data Types: Pandas has a specialized `StringDType` (`string`) vs the generic `object` type.

In [75]:
data_as_string = data.astype('string')

data_as_string

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string

In [76]:
data_as_string.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

**Slicing:** We can treat the column like a Python string.

In [77]:
# Get the first 5 characters
data_as_string.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes       <NA>
dtype: string

**Regex:** Regular expressions allow for complex pattern matching.

In [78]:
import re

In [79]:
# Pattern to identify email parts: (username) @ (domain) . (suffix)
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"

data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

**Retrieving elements:** We can chain `.str` calls to get specific parts of the regex match.

In [80]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]

matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [81]:
# Get index 1 of the tuple (the domain name)
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

The `extract` method is very powerful—it creates a new DataFrame with columns for each captured regex group.

In [82]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


### Exercise: Strings

> 1. Get the 1st group of the regex email pattern (the username).
> 2. Convert data into titlecase (e.g. "Dave@Google.Com").

## Part 3: Writing Data (Exporting)

Once your data is cleaned, you need to save it. `to_csv` is the most common method.

In [83]:
result.to_csv("../data/out.csv")

In [84]:
!cat ../data/out.csv

7558.09s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


,A,B,C
aaa,-0.264438,-1.026059,-0.6195
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


**Tip:** You usually want `index=False` to avoid saving the row numbers as a separate column.

In [89]:
result.to_csv("../data/out.csv", index=False)

In [86]:
!cat ../data/out.csv

7563.98s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


A,B,C
-0.264438,-1.026059,-0.6195
0.927272,0.302904,-0.032399
-0.264273,-0.386314,-0.217601
-0.871858,-0.348382,1.100491


**Excel Export:**

In [87]:
# Option 1: Using ExcelWriter (good for multiple sheets)
writer = pd.ExcelWriter('../data/out.xlsx')

frame.to_excel(writer, 'Sheet1')

writer.close()

/tmp/ipykernel_1876/813886760.py:4: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  frame.to_excel(writer, 'Sheet1')


In [88]:
# Option 2: Direct export
frame.to_excel('../data/out.xlsx')

---

## Optional Topics

If time permits, explore these advanced transformations.

### Renaming Axis Indexes
Changing row/column labels using mapping.

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [ ]:
data

Using `.map()` on the index:

In [ ]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

In [ ]:
# Assign back to modify in-place
data.index = data.index.map(transform)

In [ ]:
data

Using `.rename()` (returns a copy by default):

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={"OHIO": "INDIANA"}, columns={"three": 3})

### Permutation and Random Sampling
Reordering or selecting random subsets.

In [ ]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))

df

In [ ]:
# Create a random order of indices
sampler = np.random.permutation(5)

sampler

In [ ]:
# Reorder rows
df.iloc[sampler]

In [ ]:
# Alternative: use .take()
df.take(sampler)

Permuting columns:

In [ ]:
column_sampler = np.random.permutation(df.shape[1])

column_sampler

In [ ]:
df.take(column_sampler, axis=1)

**Random Sample:** Getting a random subset without manually creating a sampler.

In [ ]:
df.sample(n=3)

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])

# Sample with replacement (items can be picked more than once)
choices.sample(n=10, replace=True)

> **Task:** Sample `df` using the parameter `frac` (percentage) instead of `n` (count).

### Categorical Data
Converting string columns to `category` dtype saves memory and speeds up operations.

> **📊 Visual Illustration Available**: See ![categorical_binning](../visualisations/05_categorical_binning.png` for visualizations comparing `pd.cut()` vs `pd.qcut()`, categorical operations, and dummy variables.


In [ ]:
values = pd.Series(["apple", "orange", "apple", "apple"] * 2)

values

In [ ]:
values.unique()

In [ ]:
values.value_counts()

**Under the hood:** Categoricals are stored as integers referencing a dictionary of values.

In [ ]:
values = pd.Series([0, 1, 0, 0] * 2)

dim = pd.Series(["apple", "orange"])

values

In [ ]:
dim

In [ ]:
dim.take(values)

**Using Pandas `category` type:**

In [ ]:
fruits = ["apple", "orange", "apple", "apple"] * 2
N = len(fruits)

# to ensure reproducibility
rng = np.random.default_rng(seed=12345)

df = pd.DataFrame({"fruit": fruits, 
                   "basket_id": np.arange(N), 
                   "count": rng.integers(3, 15, size=N), 
                   "weight": rng.uniform(0, 4, size=N)}, 
                  columns=["basket_id", "fruit", "count", "weight"])

df

In [ ]:
# Convert object column to category
fruit_cat = df['fruit'].astype('category')

fruit_cat

The values for `fruit_cat` are now an instance of `pandas.Categorical`, which you can access via the `.array` attribute:

In [ ]:
c = fruit_cat.array

type(c)

In [ ]:
c.categories

In [ ]:
c.codes

In [ ]:
dict(enumerate(c.categories))

In [ ]:
# Assign back to the dataframe
df['fruit'] = df['fruit'].astype('category')

df

**Creating from Codes:**

In [ ]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

my_cats = pd.Categorical.from_codes(codes, categories)

my_cats

**Ordered Categoricals:** Useful for Likert scales or sizes (Small < Medium < Large).

In [ ]:
ordered_cats = pd.Categorical.from_codes(codes, categories, ordered=True)

ordered_cats

**Binning Data (`pd.cut`):** Converting continuous data (e.g., Age) into categorical bins (e.g., Age Groups).

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [ ]:
# Define bin edges
bins = [18, 25, 35, 60, 100]

# Cut the data
age_cat = pd.cut(ages, bins)

age_cat

In [ ]:
age_cat.codes

In [ ]:
age_cat.categories

In [ ]:
age_cat.categories[0]

In [ ]:
pd.value_counts(age_cat)

**Binning Parameters:**

In [ ]:
# right=False makes the left side closed [inclusive, exclusive)
pd.cut(ages, bins, right=False)

In [ ]:
# Adding custom labels
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]

pd.cut(ages, bins, labels=group_names)

If you pass an integer number of bins to `pd.cut`, it will compute equal-length bins based on the min/max of the data.

In [ ]:
data = np.random.uniform(size=20)

pd.cut(data, 4, precision=2)

> **Task:** Cut `ages` into 5 bins instead of 4. Set the labels to `['Youth', 'YoungAdult', 'MiddleAged', 'Senior', 'Elderly']`.

**Categorical Methods (.cat):**

In [ ]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')

cat_s

In [ ]:
cat_s.cat.codes

In [ ]:
cat_s.cat.categories

Modifying categories:

In [ ]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)

cat_s2

In [ ]:
cat_s.value_counts()

In [ ]:
cat_s2.value_counts()

In [ ]:
cat_s2.cat.remove_unused_categories()

**Computing Indicator / Dummy Variables:** Converting categorical variables into binary columns (One-Hot Encoding) for machine learning.

In [ ]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], 
                   "data1": range(6)})

df

In [ ]:
pd.get_dummies(df["key"])

In [ ]:
dummies = pd.get_dummies(df["key"], prefix="key")

dummies

Recipe: Combining `get_dummies` with `cut`.

In [ ]:
np.random.seed(12345)

values = np.random.uniform(size=10)

values

In [ ]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

pd.get_dummies(pd.cut(values, bins))

> **Task:** Cut values into 4 bins and create dummy variables.

### Databases

Connecting to a database using `sqlalchemy`.

In [ ]:
import sqlalchemy as sqla

In [ ]:
import os 

parent_dir = os.path.abspath(os.path.pardir)

In [ ]:
# Create connection string
engine = sqla.create_engine(f'duckdb:///{parent_dir}/data/unit-1-4.db')

In [ ]:
# Read entire table
df = pd.read_sql('resale_flat_prices_2017', engine)

In [ ]:
df

In [ ]:
# Execute raw SQL query
df = pd.read_sql("SELECT * FROM resale_flat_prices_2017", engine)

In [ ]:
engine.table_names()

### Binary files (Pickle)

`pickle` is Python's native serialization format. Good for short-term storage, but not for sharing data between different languages.

In [ ]:
result.to_pickle('../data/out.pkl')

In [ ]:
data = pd.read_pickle('../data/out.pkl')

data

> **Task:** Write a filtered DataFrame to a new database table.
> 1. Filter for flats in "YISHUN".
> 2. Write to table `yishun_flat_prices_2017`.

In [ ]:
df_yishun = df[df.town == "YISHUN"]

df_yishun.to_sql("yishun_flat_prices_2017", engine)

> **Final Challenge:** 
> 1. Read only flats from `BISHAN` to a new dataframe.
> 2. Write the dataframe to a new database table `bishan_flat_prices_2017`.

### Post Class

> **Question 1**
> Question: How do you select rows from a DataFrame where any value in the row exceeds a threshold?

In [91]:
import pandas as pd

df = pd.DataFrame({'A': [1, 2, 3, 4, 5], 'B': [10, 20, 30, 40, 50]})
threshold = 30

# 1. Create a boolean mask of the whole DataFrame
mask = df > threshold

# 2. Use .any(axis=1) to find rows where at least one column is True
rows_exceeding = df[mask.any(axis=1)]

print(rows_exceeding)


   A   B
3  4  40
4  5  50


In [ ]:
> **Question 2**
> Question: How do you sort a DataFrame by columns A and B?

In [93]:
import pandas as pd

df = pd.DataFrame({'A': [2, 1, 2], 'B': [2, 3, 1], 'C': [1, 2, 3]})

df_sorted = df.sort_values(by=['A','B'])

print(df_sorted)


   A  B  C
1  1  3  2
2  2  1  3
0  2  2  1


In [ ]:
> **Question 3**
> Question: How do you concatenate two DataFrames vertically?

In [95]:
import pandas as pd

df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [7, 8, 9], 'B': [10, 11, 12]})

# Concatenate vertically
result = pd.concat([df1, df2])

print(result)

   A   B
0  1   4
1  2   5
2  3   6
0  7  10
1  8  11
2  9  12


In [ ]:
> **Question 4**
> Question: How do you compute the cumulative sum of a column in a DataFrame?

In [96]:
import pandas as pd

df = pd.DataFrame({'A': [1, 2, 3, 4, 5]})

# Create a new column for the cumulative sum
df['Cumulative_Sum'] = df['A'].cumsum()

print(df)

   A  Cumulative_Sum
0  1               1
1  2               3
2  3               6
3  4              10
4  5              15


In [ ]:
> **Question 5**
> Question: How do you convert a Series of strings to uppercase?

In [97]:
import pandas as pd

series = pd.Series(['apple', 'banana', 'cherry'])

# Convert the entire series to uppercase
uppercase_series = series.str.upper()

print(uppercase_series)

0     APPLE
1    BANANA
2    CHERRY
dtype: object
